# Advanced Plotting in Seaborn and How to Use it in EDA
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp25&branch=main&urlpath=tree%2Fdata271_sp25%2Flectures%2Fdata271_lec22_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import covid data
covid = pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Python-Data-Cleaning-Cookbook/master/Chapter05/data/covidtotals.csv')

### Initial inspection

In [ ]:
covid.head()

In [ ]:
covid.shape

In [ ]:
covid.info()

`covid` is a DataFrame with with 209 rows. Each row represents a location (country). It contains the following columns: 

- iso_code - a unique 3 digit identifier for each country
- lastdate - the date for when the report was last updated
- location - full name of the country
- total_cases - the cumulative number of confirmed COVID-19 cases in that location
- total_deaths - the cumulative number of confirmed COVID-19 deaths in that location
- total_cases_pm - the cumulative number of confirmed COVID-19 cases per million in that location
- total_deaths_pm - the cumulative number of confirmed COVID-19 deaths per million in that location
- population - population size
- pop_density - number of people per square kilometer
- median_age - median age of the population 
- gdp_per_capita - The GDP per capita (economic indicator of wealth per person)
- hosp_beds - The number of hospital beds per 1,000 people (an indicator of healthcare capacity)
- region - broader region 

### Preprocessing

In [ ]:
# Check for na values
covid.isna()

In [ ]:
# Summarize number of na's by column
nas = covid.isna().sum()
nas

In [ ]:
# drop columns with na values
covid.dropna(axis=1)

In [ ]:
covid.shape

In [ ]:
# drop rows with na values
covid.dropna(axis=0,inplace=True)

In [ ]:
covid.shape

In [ ]:
# check if there are any duplicate rows
covid.duplicated().sum()

In [ ]:
# check the data types to make sure they are appropriate
covid.dtypes

In [ ]:
covid['lastdate'] = pd.to_datetime(covid['lastdate'])
covid.dtypes

### Summary statistics and Exploration

In [ ]:
# begin exploring the dataset
covid_desc = covid.describe()
covid_desc

In [ ]:
# What is the average death rate by region?
covid.groupby('region')['total_deaths_pm'].mean()

In [ ]:
# Which country has the highest death counts
covid.loc[covid.total_deaths.idxmax(),'location']

In [ ]:
# Which country has the highest death rate
covid.loc[covid.total_deaths_pm.idxmax(),'location']

# Visualization

In [ ]:
# heatmaps to look at correlations
corrmatc = covid.corr(numeric_only = True) # make correlation matrix
sns.heatmap(data = corrmatc, annot = True);

The median age and gdp per capita are correlated with deaths per million. This is interesting. 

In [ ]:
# Let's look at the relationship with gdp
sns.scatterplot(data = covid, x = 'gdp_per_capita',y = 'total_deaths_pm');

In [ ]:
# Where would the regression line be?
sns.regplot(data = covid, x = 'gdp_per_capita',y = 'total_deaths_pm');

In [ ]:
# That's a little hard to see
sns.jointplot(data = covid, x = 'gdp_per_capita',y = 'total_deaths_pm');

In [ ]:
# boxplots to look at deaths by region
plt.figure(figsize=(4,6))
sns.boxplot(y = covid.region,x=covid.total_deaths_pm)
plt.show()

There are a few regions we might want to zoom in on...

In [ ]:
high_covid_regions = covid.loc[covid.region.isin(['South America','Western Europe','North America'])]
high_covid_regions.head()

In [ ]:
# Let's redo the boxplot with these points
sns.boxplot(data = high_covid_regions, y = 'region',x='total_deaths_pm');

In [ ]:
# To get more information about the distributions
sns.violinplot(data = high_covid_regions, y = 'region',x='total_deaths_pm');

In [ ]:
# Show all the points in the distributions with a swarm plot
sns.swarmplot(data = high_covid_regions, y = 'region',x='total_deaths_pm');

In [ ]:
# Notice a swarm plot is different from a scatter plot
sns.scatterplot(data = high_covid_regions, y = 'region',x='total_deaths_pm');

In [ ]:
# Another way to compare categories (best if this is an ordinal variable)
sns.pointplot(data = high_covid_regions, x = 'region',y='total_deaths_pm');

In [ ]:
# Let's look at that gdp relationship again
sns.jointplot(data = high_covid_regions, x = 'gdp_per_capita',y = 'total_deaths_pm', hue = 'region');
plt.xlim([0,120000]);
plt.ylim([-20,1000]);

South America has almost a bimodal distribution

In [ ]:
# Focus on these distributions more (messy!)
sns.histplot(high_covid_regions, x = 'total_deaths_pm', hue = 'region');

In [ ]:
# Better
sns.kdeplot(high_covid_regions, x = 'total_deaths_pm', hue = 'region', fill=True);

In [ ]:
# Even better 
g = sns.FacetGrid(high_covid_regions, col="region", hue = 'region')
g.map(sns.kdeplot, "total_deaths_pm", fill = True)
plt.tight_layout();

In [ ]:
na = covid.loc[covid.region == 'North America', :]
na

In [ ]:
# A teaser for interactive plots

import plotly.express as px


fig = px.scatter(covid, x = 'gdp_per_capita', y = 'total_deaths_pm', hover_name='location')
# Set figure size
fig.update_layout(width=600, height=600)